In [1]:
import pandas as pd
import numpy as np

In [2]:
data_q1 = pd.read_excel('comp1.xlsx')
data_q2 = pd.read_excel('comp2.xlsx')

# Peak at Data 

In [3]:
data_q1.head()

,Start,End,Duration,2018 - Q1,Vert. Position,Vert. Line,Comment
0,2018-03-28,2018-04-11,15,Task 1\nMar 28 - Apr 11,-25,-25,Done
1,2018-04-12,2018-05-01,20,Task 2\nApr 12 - May 1,-40,-15,Done
2,2018-05-02,2018-05-25,24,Task 3\nMay 2 - May 25,-55,-15,Done
3,2018-04-28,2018-07-12,76,Task Label 4,-80,-80,Done
4,2018-06-15,2018-07-04,20,Task Label 5,-30,-30,TBC


In [4]:
data_q2.head()

,Start,End,Duration,2018 - Q2,Vert. Position,Vert. Line,Comment
0,2018-03-28,2018-04-11,15,Task 1\nMar 28 - Apr 11,-25,-25,Done
1,2018-04-12,2018-05-01,20,Task 2\nApr 12 - May 1,-40,-15,Done
2,2018-05-02,2018-06-25,55,Task 3\nMay 2 - Jun 25,-55,-15,TBD
3,2018-04-28,2018-06-21,55,Need info,-80,-80,TBD
4,2018-06-15,2018-07-04,20,Task Label 5,-30,-30,TBC


# Data Cleaning/Standardize

In [5]:
data_q1['2018 - Q1'] = data_q1['2018 - Q1'].str.replace('\n', ' ')
data_q2['2018 - Q2'] = data_q2['2018 - Q2'].str.replace('\n', ' ')

In [6]:
data_q1.rename(columns={'2018 - Q1':'2018'}, inplace= True)
data_q2.rename(columns={'2018 - Q2':'2018'}, inplace= True)

In [7]:
data_q1.head()

,Start,End,Duration,2018,Vert. Position,Vert. Line,Comment
0,2018-03-28,2018-04-11,15,Task 1 Mar 28 - Apr 11,-25,-25,Done
1,2018-04-12,2018-05-01,20,Task 2 Apr 12 - May 1,-40,-15,Done
2,2018-05-02,2018-05-25,24,Task 3 May 2 - May 25,-55,-15,Done
3,2018-04-28,2018-07-12,76,Task Label 4,-80,-80,Done
4,2018-06-15,2018-07-04,20,Task Label 5,-30,-30,TBC


In [19]:
data_q2.head()

,Start,End,Duration,2018,Vert. Position,Vert. Line,Comment
0,2018-03-28,2018-04-11,15,Task 1 Mar 28 - Apr 11,-25,-25,Done
1,2018-04-12,2018-05-01,20,Task 2 Apr 12 - May 1,-40,-15,Done
2,2018-05-02,2018-06-25,55,Task 3 May 2 - Jun 25,-55,-15,TBD
3,2018-04-28,2018-06-21,55,Need info,-80,-80,TBD
4,2018-06-15,2018-07-04,20,Task Label 5,-30,-30,TBC


In [8]:
data_q1.shape[1] # columns to compare

7

# Merge Inner

In [9]:
data_inner = data_q1.merge(data_q2, how= 'outer', indicator= True)

In [10]:
# There is a diffrence between these rows, but we don't know exactly(column) where diffrence is located. 
data_inner.loc[data_inner['_merge'] != 'both'].reset_index(drop='index')

,Start,End,Duration,2018,Vert. Position,Vert. Line,Comment,_merge
0,2018-05-02,2018-05-25,24,Task 3 May 2 - May 25,-55,-15,Done,left_only
1,2018-04-28,2018-07-12,76,Task Label 4,-80,-80,Done,left_only
2,2018-09-11,2018-09-15,5,Task Label 8,-60,-40,TBC,left_only
3,2018-09-16,2018-10-06,21,Task Label 9,-75,-15,TBC,left_only
4,2018-05-02,2018-06-25,55,Task 3 May 2 - Jun 25,-55,-15,TBD,right_only
5,2018-04-28,2018-06-21,55,Need info,-80,-80,TBD,right_only
6,2018-09-11,2018-09-15,5,Moved to q3,-60,-40,Done,right_only
7,2018-09-16,2018-10-06,21,Task Label 9,-75,-15,Done,right_only


# Show where the difference are 

In [11]:
compare = pd.concat([data_q1,data_q2])

In [12]:
diff = data_q1 == data_q2

In [13]:
diff.replace({False: 'X', True: ' '}, inplace=True)

In [14]:
# These are the location where the diffrence are loacted. ROW{0-4 = 1st data}, ROW{4-8 = 2nd data}
diff

,Start,End,Duration,2018,Vert. Position,Vert. Line,Comment
0,,,,,,,
1,,,,,,,
2,,X,X,X,,,X
3,,X,X,X,,,X
4,,,,,,,
5,,,,,,,
6,,,,,,,
7,,,,X,,,X
8,,,,,,,X


# Show exactly where the diffences are located

In [15]:
df_mask = data_q1.compare(data_q2, keep_shape=True).notnull().astype('int').rename(columns={'self': 'Data_q1', 'other': 'Data_q2'}, level=-1)
df_compare = data_q1.compare(data_q2, keep_shape=True, keep_equal=True).rename(columns={'self': 'Data_q1', 'other': 'Data_q2'}, level=-1)

In [16]:
def apply_color(x):
    colors = {1: 'lightyellow', 0: 'white'}
    return df_mask.applymap(lambda val: 'background-color: {}'.format(colors.get(val,'')))

data_compare = df_compare.style.apply(apply_color, axis=None)

In [17]:
data_compare

In [18]:
# Export the data into excel and highlight the difference between data1 and data2
data_compare.to_excel('comp.xlsx')